In [5]:
import healpy as hp
import numpy as np
import burstutils
import random as rand
import statistics as s


This is my workspace where I'm experimenting with the class functions before I end up transferring it into the py version. Some of the things I still need to do while here is:


Documentation

Random gauss vs poisson

Importing modules 

 adding skymaps. 
 
 
 Updating how classes are defined (input really best approach?) 

After redefining classes, build up BurstCube class so that it runs the entire sim! call on module that contains all the functions, length, angle, sep, etc. Should be easiest this way

In [1]:
class GRBs():
   #this produces an error, deal with soon!
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    
  #  import numpy as np
   # import healpy as hp
    def __init__(self,NSIDE,strength):
        from healpy import nside2npix
        from healpy import pix2ang
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(pix2ang(NSIDE,i))

    def say_Ao(self):
        print("The GRBs being tested will be " + str(self.Ao) + " counts strong.")
        
            

In [2]:
GRB = GRBs(4,500)

In [3]:
GRB.pixels

192

In [4]:
GRB.say_Ao()

The GRBs being tested will be 500 counts strong.


In [5]:
class BurstCube():

    def __init__(self,background):
        import numpy as np
        import healpy as hp
        import burstfuncs as bf
        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
    
    #make the normal vectors!
        #self.detA = [ zenith[0] + self.tiltA , zenith[1] ] 
        #self.detB = [ zenith[0] + self.tiltB , zenith[1] + np.pi/2 ] 
       # self.detC = [ zenith[0] + self.tiltC , zenith[1] + np.pi ] 
        #self.detD = [ zenith[0] + self.tiltD , zenith[1] + 3*np.pi/2 ] 

        #self.Anorm = hp.ang2vec(self.detA[0],self.detA[1])
        #self.Bnorm = hp.ang2vec(self.detB[0],self.detB[1])
        #self.Cnorm = hp.ang2vec(self.detC[0],self.detC[1])
        #self.Dnorm = hp.ang2vec(self.detD[0],self.detD[1])

    
       # self.dets = [self.Anorm,self.Bnorm,self.Cnorm,self.Dnorm] 
        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + np.pi]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    def response2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
       
        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        
        self.localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
            print("Testing " + str(np.rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<5:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(sourcexyz,self.dets[j])  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
                
                
                coarsethetaloc,coarsephiloc,coarseAo = bf.solver(detcounts,self.dets,0,90,0,360,10,37,self.bg)
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,coarsethetaloc-5,coarsethetaloc+5,coarsephiloc-5,coarsephiloc+5,10,10,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
              #  print("Recovered position at " + str(recpos))
                
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc: " + str(np.rad2deg(bf.angle(sourcexyz,recvec))) + " deg")
                loop+=1
          #  print("Obtained avg loc unc of " + np.rad2deg(s.mean(locunc)) + "")
            if len(locunc)>0:
                self.localizationerrors.append(np.rad2deg(s.mean(locunc)))
            else:
                self.localizationerrors(1000)
                #some big # signifying that it can't catch it 

            print("Loc unc " +  str(np.rad2deg(s.mean(locunc))))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        
        maybe = input("All done with the simulation, want to make the skymap too? ")
        
        if maybe == "yes" or maybe == "Yes" or maybe == "y": 
            import matplotlib.pyplot as plt
            from healpy import newvisufunc
            if len(self.localizationerrors) == len(GRB.sourceangs):
                im = np.array(self.localizationerrors)
            else:
                blockedpart=1000*np.ones(GRB.pixels-len(self.localizadtionerrors))
                im = np.concatenate((self.localizationerrors,blockedpart))
            hp.newvisufunc.mollview(im,min=0, max=30,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
            plt.title('All Sky Localization Uncertainty for BurstCube')
        else:
            print("Ok, maybe next time!")

        return self.localizationerrors
            
    
    

In [ ]:
    def plot_skymap(self,GRB):
        import numpy as np
        import matplotlib.pyplot as plt
        import healpy as hp
        from healpy import newvisufunc
        if len(self.localizationerrors) == len(GRB.sourceangs):  #if the function successfully was able to catch all the spots (meaning nothing went wrong!)
            im = np.array(self.localizationerrors) 
        else:
            blockedpart=1000*np.ones(GRB.pixels-len(angoffset))
            im=np.concatenate((self.localizationerrors,blockedpart))
        hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')
        plt.title('All Sky Localization Uncertainty for BurstCube')
        

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
from healpy import newvisufunc
%matplotlib notebook
if len(im) == 48:
    im = np.array(im)
else:
    blockedpart= 1000*np.ones(48-len(im))
    im=np.concatenate((im,blockedpart))
hp.newvisufunc.mollview(im,min=0, max=50,unit='Localization Accurary (degrees)',graticule=True,graticule_labels=True,cmap='viridis_r')


In [ ]:
im = [12.193725149835284,
 25.25448240996645,
 22.305707106859245,
 23.965693073949609,
 52.008548584668084,
 27.821629706579458,
 37.423685868114056,
 68.476263273719425,
 30.625361189430574,
 23.401815051293706,
 43.663123284390203,
 19.564729694837329,
 51.433859688574401,
 74.844783683652736,
 56.576057682336121,
 33.323870897313476,
 90.612650101315353,
 82.253008561973189,
 80.329406453087046,
 94.19026553811679,
 99.538349117642525,
 75.175862782099529,
 51.048488219348087,
 74.128524449751453,
 58.480977383443751,
 52.597647136484362,
 77.222734640119555,
 103.61494510174424,
 41.109554839392487,
 76.269478377179141,
 98.202392939060829,
 99.73653357751094,
 122.38781518900511,
 74.733648939874953,
 99.483079370711494,
 100.81648250231294,
 142.5793551632658,
 128.99519400895949,
 109.03771544592294,
 98.612082515419004,
 127.9138150277825,
 93.497546429808551,
 141.23279694489702,
 130.06560240843098,
 157.61167766859899,
 129.80910207180099]


In [ ]:
class Plots:

In [6]:
cube1 = BurstCube(background=1000)

ModuleNotFoundError: No module named 'burstfuncs'

In [ ]:
cube1.response2GRB(GRB)

In [ ]:
cube1.plot_skymap()

In [ ]:
import matplotlib.pyplot as plt
plt.show()

In [ ]:
#to generate an array of all the source angles, and basically follow that 

GRB = GRBs(2)  #only thing I have to identify is the nside
len(GRB.sourceangs)

In [ ]:
import numpy as np
import healpy as hp
import burstfuncs as bf
import random as rand

class GRBs():
 
    """
    Generates an array of GRB's with some given strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):
        import numpy as np
        import healpy as hp
        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = hp.nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(hp.pix2ang(NSIDE,i))


class BurstCube:
    def __init__(self,background):
        import numpy as np
        import healpy as hp
        import burstfuncs as bf
        self.zenith = [0 , 0]
        self.bg = background
        test = input("Are the detectors alternating? ")
        if test == "yes" or test == "Yes" or test == "y" or test == "asdlfkjawe":           
            self.tiltA = np.deg2rad(float(input("Please enter the first tilt (deg) ")))
            self.tiltB = np.deg2rad(float(input("Please enter the second tilt (deg) ")))
            self.tiltC = self.tiltA
            self.tiltD = self.tiltB
        else:
            self.tilt = np.deg2rad(float(input("Please enter the tilt (deg) ")))
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
            
        
    @property
    def detA(self):
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + np.pi/2 ]
    @property
    def detC(self):
        return [ self.zenith[0] + self.tiltC , self.zenith[1] ]
    @property 
    def detD(self):
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + np.pi/2 ]
    @property
    def normA(self):
        return  hp.ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  hp.ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  hp.ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  hp.ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 
    
    
    
    
    def respond2GRB(self, GRB):   #is this how I inherit? 
        #first need to include the GRB.
        
        #Putting these here as a reminder, need to learn how to implement Gaussian fluctuations, hopefully in a similar to the actions gauss(x,sigmax)
        #Gauss = True
        #Poisson = False


        """
        Using least squares regression, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
        
        An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 

        Returns
        ----------
        localizationerrors : array

        numpy array that contains the average localization uncertainty at each sky position. 

        
        """
        localizationerrors = []    
        for i in range(len(GRB.sourceangs)):
            sourceAng = GRB.sourceangs[i]
           # print("Source angle is " + str(sourceAng))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = hp.ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
            loop = 0 #I'm going to want to sample each sky position more than once,
                    #here's where I define how many times that is
            locunc = []
            while loop<3:
                detcounts = []  #number of counts incident in each detector. 
                for j in range(len(self.dets)):
                    sep=bf.angle(sourcexyz,self.dets[j])
                   # print("separation is " + str(np.rad2deg(sep)))
                   #this check passes.  
               
                    if sep < np.pi/2: # meaning if >90, would not be facing detector.
                        dtheory=GRB.Ao*bf.response(sourcexyz,self.dets[j])  #still need to define strength, brb and gonna do that 
                    else: #like I was saying, has to face it!
                        dtheory = 0 
                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    
                    counts = dtheory + self.bg #another artifact, incl this background effect somewhere
                    unccounts = np.sqrt(counts)
                    detactual = rand.gauss(counts,unccounts)  #there is a lot of noise, present, updating it now. 
                    if detactual-self.bg < 0:
                        detactual = 0
                    
                    detcounts.append(detactual)
    
                   #now for the LSF method using chi squared, applies a coarse to fine refactor here. 
                   #plenty of old variable names to update as well. 
                
                   # print("after fluctuations, " + str(detactual))
                    #fixed indentation errors, uninstalled module, legacy code all fixed. Passes. 

                
                
                coarsethetaloc,coarsephiloc,coarseAo = bf.solver(detcounts,self.dets,0,90,0,360,12,self.bg)
                finethetaloc,finephiloc,fineAo = bf.solver(detcounts,self.dets,coarsethetaloc-5,coarsethetaloc+5,coarsephiloc-7,coarsephiloc+7,12,self.bg)
                
                if finethetaloc > 180:
                #    print("it recovered an unrealistic answer, skip")
                    break
                elif finethetaloc < 0:
                    #print("Same issue, there are limits to theta that are broken here. ")
                    break 
                    
                recpos = [finethetaloc,finephiloc]
                recvec = hp.ang2vec(np.deg2rad(finethetaloc),np.deg2rad(finephiloc))
                locunc.append(bf.angle(sourcexyz,recvec))
               # print("loc unc" + str(locunc))
                loop+=1
            print(np.rad2deg(s.mean(locunc)))

            localizationerrors.append(np.rad2deg(s.mean(locunc)))

           # print("obtained error of" +  str(s.mean(locunc)))
                #loc unc is the uncertainty at each sky position, localerros is for all of them
            
            #should there be more selfs? more prints? basically done so time to debug after lunch. 
        return localizationerrors
        

In [ ]:
GRB = GRBs(NSIDE=8,strength=500)

In [ ]:
cubesat = BurstCube(background=1000)

In [ ]:
cubesat.bg

In [ ]:
cubesat.respond2GRB(GRB)

In [7]:
import numpy as np

In [9]:
type(np.arange(0,100))


numpy.ndarray

In [10]:
type(np.linspace(0,100,101))

numpy.ndarray

In [ ]:
        
        Chisep = 


        
        for sa in range(len(oa)): 
            for sb in range(len(ob)):
                for sc in range(len(Aofit)):
                    #make sure it fits within the acceptable range
                    if oa[sa]>=0 and oa[sa]<=np.pi and ob[sb]>=0 and ob[sb]<=2*np.pi:
                        CHIsourceang=[oa[sa],ob[sb]]
                        CHIsourcexyz = hp.ang2vec(CHIsourceang[0],CHIsourceang[1])
                        CHIsep=angle(CHIsourcexyz,detnorms[s])                                           
                        if CHIsep<np.pi/2: 
                            chi=Aofit[sc]*response(CHIsourcexyz,detnorms[s])+bgrd
                            #print("Chi test angle"+str(CHIsourcexyz))
                            #print("detector"+str(dets[s]))
                            #print("chi sometiems"+str(chi))
                            #print("separation here, is it okay? " +str(np.rad2deg(CHIsep)))

                            #this produces nan error, se
                            
                        else:
                            chi=0            
                        if detsvals[s]>0:   #if there is a signal in the detector 
                            chiterm=((chi-detsvals[s])**2/detsvals[s])
                        else:    #if not, just zero 
                            chiterm=1e10

In [8]:
from astropy.coordinates import SkyCoord

In [11]:
SkyCoord.separation(normA,source)

AttributeError: 'list' object has no attribute 'is_equivalent_frame'

In [9]:
normA = [np.pi/4,0]

In [10]:
source = [0,0]

In [ ]:
def solver(detsvals,detnorms,bottheta,toptheta,botphi,topphi,ntheta,nphi,bgrd):
#this function uses a chi squared minimizer over a given range to identify the theta,phi,
#and Ao values which correspond to the minimum chi squared and thus localized source. 
    confidence = []
    chiterms = []
    thecon = []
    phicon = []
    for s in range(len(detsvals)):     
        
 
                    if oa[sa]>=0 and oa[sa]<=np.pi and ob[sb]>=0 and ob[sb]<=2*np.pi:
                        CHIsourceang=[oa[sa],ob[sb]]
                        CHIsourceyz = hp.ang2vec(CHIsourceang[0],CHIsourceang[1])
                        CHIsep=angle(CHIsourcexyz,detnorms[s])                                           
                        if CHIsep<np.pi/2: 
                            c
                            
                            chi=Aofit[sc]*response(CHIsourcexyz,detnorms[s])+bgrd
                            #print("Chi test angle"+str(CHIsourcexyz))
                            #print("detector"+str(dets[s]))
                            #print("chi sometiems"+str(chi))
                            #print("separation here, is it okay? " +str(np.rad2deg(CHIsep)))

                            #this produces nan error, se
                            
                        else:
                            chi=0            
                        if detsvals[s]>0:   #if there is a signal in the detector 
                            chiterm=((chi-detsvals[s])**2/detsvals[s])
                        else:    #if not, just zero 
                            chiterm=1e10
                    else: 
                        chiterm=1e10 #some large #, just to note that it definitely isn't right angle 
                                                                       
                                    
                    chiterms.append(chiterm)   #this is an array of EVERY SINGLE term, need to split in pieces and add element by element...
    chisquareds=chimaker(chiterms,len(detsvals))
    chimin=np.amin(chisquareds)
    #print("The chi min is " + str(chimin))
    #is there a better way to do this? 
    chisquareds=list(chisquareds)

    thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
    philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
    #print(Aoguess)
    #print(thetaloc,philoc)
    
    return thetaloc,philoc,Aoguess


In [23]:
import numpy as np
import time 

In [51]:
start = time.time()
theta = np.deg2rad(np.linspace(0,90,100))
phi = np.deg2rad(np.linspace(0,360,100))
A = np.linspace(0,1000,11)
chi=A*response(CHIsourcexyz,detnorms[s])+bgrd

xyz = [theta,phi]

In [52]:
theta

array([ 0.        ,  0.17453293,  0.34906585,  0.52359878,  0.6981317 ,
        0.87266463,  1.04719755,  1.22173048,  1.3962634 ,  1.57079633])

In [108]:
import healpy as hp
start = time.time()
theta = np.deg2rad(np.linspace(0,90,1000))
phi = np.deg2rad(np.linspace(0,360,1000))
A= np.linspace(0,1000,1000)
CHIsourcexyz = hp.ang2vec(theta,phi)
fakenorm = [0,0,1] #need to make this an array really long

chisep = []
for i in range(len(CHIsourcexyz)): #all 1000 anyway
    chisep.append(angle(CHIsourcexyz[i],fakenorm))
    
chisep = np.array(chisep) 

chiresponse =  A * np.cos(chisep) #DONT include background, its subtracted anyway... it isn't in the code but lets talk about this first. 

end = time.time()
print("Time elapsed: " + str(end-start))

Time elapsed: 0.012300968170166016


In [116]:
chiterms = []
for i in range(len(chiresponse)):
    chiterm = (actual - chiresponse[i])**2/ actual 
    chiterms.append(chiterm)


In [121]:
chimin = min(chiterms)

In [124]:
thetaloc = np.rad2deg(theta[int((chiterms.index(chimin)-(chiterms.index(chimin) % (len(phi)*len(A))))/(len(phi)*len(A)))])
philoc = np.rad2deg(phi[int(((chiterms.index(chimin) % (len(phi)*len(A)))-(chiterms.index(chimin) % (len(A))))/len(A))])

In [125]:
thetaloc,philoc

(0.0, 0.0)

In [96]:
type(A)

numpy.ndarray

In [87]:
   
    
    if chisep> np.pi/2:
    
    chiresponse =  A * np.cos(chisep)   
else:
    
#need to create an array! 
end = time.time()
print("Time elapsed: " + str(end-start))

Time elapsed: 0.018271923065185547


In [90]:
min(chiresponse)

0.0

In [80]:
a = []
for i in range(1000):
    a.append(fakenorm)


    
chiterm = A * np.cos(angle(CHIsourcexyz,a))
end = time.time()
print("Time elapsed: " + str(end-start))

ValueError: shapes (1000,3) and (1000,3) not aligned: 3 (dim 1) != 1000 (dim 0)

In [78]:
#so the error to resolve now is in the angle function, the issue is something with the shapes not being aligned.

ValueError: shapes (1000,3) and (1000,3) not aligned: 3 (dim 1) != 1000 (dim 0)

In [29]:
def angle(v1, v2):
    """"
    Finds the angle between 2 vectors
    
    Parameters
    ----------
    
    v1 : array
    v2 : array
        The arrays representing the vectors who's angle is to be calculated.
        
    Returns
    -------
    
    ang : float
        Angle between the 2 vectors. 
        
    """

    ang = np.arccos(np.dot(v1, v2) / (length(v1) * length(v2)))
    return ang


for i in range(len(theta)):
    for j in range(len(phi)):
        for k in range(len(A)):
            if i ==600 and j ==3000 and k == 600:
                print("Done")

end = time.time()
print("Time elapsed: " + str(end-start))


#combine all of these, creates an array of 4070!

Time elapsed: 0.0006878376007080078


In [63]:
def angle(v1, v2):
    """"
    Finds the angle between 2 vectors
    
    Parameters
    ----------
    
    v1 : array
    v2 : array
        The arrays representing the vectors who's angle is to be calculated.
        
    Returns
    -------
    
    ang : float
        Angle between the 2 vectors. 
        
    """

    ang = np.arccos(np.dot(v1, v2) / (length(v1) * length(v2)))
    return ang

def length(v):
    """
    Finds the length of a vector
    
    Parameters
    ----------
    
    v : array
        numpy array representative of the vector you want to find the magnitude of. 
    
    Returns
    -------
    
    magv : float
        magnitude of v.
    """
    magv = np.sqrt(np.dot(v, v))
    return magv

(array([  0.,  10.,  20.,  30.,  40.,  50.,  60.,  70.,  80.,  90.]),
 array([ 20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.,  20.]))

In [ ]:
def solver(detsvals,detnorms,bgrd):
#this function uses a chi squared minimizer over a given range to identify the theta,phi,
#and Ao values which correspond to the minimum chi squared and thus localized source. 
    """Stray thoughts about code so far. Should be quicker, rather than 1000 use arbitrary vals, as well as top and bottom phi
    No longer need chimaker since it adds it in. 
    """
    
    chiterms = np.zeros(1000)
    confidence = []
    chiterms = []
    thecon = []
    phicon = []
    for s in range(len(detsvals)):     
        detresponse = detsvals[s]*np.ones(1000)
        oa = np.deg2rad(np.linspace(0,90,1000))
        ob = np.deg2rad(np.linspace(0,360,1000))
        Ao = np.linspace(0,1000,1000)
        bg = bgrd*np.ones(1000)
        CHIsourcexyz = hp.ang2vec(oa,ob)
        fakenorm = [0,0,1] #need to make this an array really long

        chisep = []
        for i in range(len(CHIsourcexyz)): #all 1000 anyway
            chisep.append(angle(CHIsourcexyz[i],fakenorm))
        
        #If necessary,here's where I would create another array saying to ignore >pi/2 terms.
        chisep = np.array(chisep) 

        chiresponse =  Ao * np.cos(chisep) + bg  
        chiterm=((chi-detsvals[s])**2/detsvals[s])
        chiterms += np.divide((chirespone-detresponse)**2,detresponse)

                                                                       
        

    chimin=np.amin(chisquareds)

    chisquareds=list(chisquareds)

    thetaloc = np.rad2deg(oa[int((chisquareds.index(chimin)-(chisquareds.index(chimin) % (len(ob)*len(Aofit))))/(len(ob)*len(Aofit)))])
    philoc = np.rad2deg(ob[int(((chisquareds.index(chimin) % (len(ob)*len(Aofit)))-(chisquareds.index(chimin) % (len(Aofit))))/len(Aofit))])
    Aoguess=Aofit[int((chisquareds.index(chimin) % (len(ob)*len(Aofit)))  % len(Aofit))]
    #print(Aoguess)
    #print(thetaloc,philoc)
    
    return thetaloc,philoc,Aoguess


In [6]:
baba = 1000*np.ones(10)

oba = np.linspace(0,10,10)

In [2]:
import numpy as np

In [7]:
oba + baba

array([ 1000.        ,  1001.11111111,  1002.22222222,  1003.33333333,
        1004.44444444,  1005.55555556,  1006.66666667,  1007.77777778,
        1008.88888889,  1010.        ])

In [8]:
(oba - baba)**2

array([ 1000000.        ,   997779.01234568,   995560.49382716,
         993344.44444444,   991130.86419753,   988919.75308642,
         986711.11111111,   984504.9382716 ,   982301.2345679 ,   980100.        ])

In [9]:
oba

array([  0.        ,   1.11111111,   2.22222222,   3.33333333,
         4.44444444,   5.55555556,   6.66666667,   7.77777778,
         8.88888889,  10.        ])

In [10]:
baba

array([ 1000.,  1000.,  1000.,  1000.,  1000.,  1000.,  1000.,  1000.,
        1000.,  1000.])

In [11]:
abba = np.linspace(3,20,10)

In [16]:
np.divide(abba,oba)

/Users/noahkasmanoff/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:1: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


array([        inf,  4.4       ,  3.05      ,  2.6       ,  2.375     ,
        2.24      ,  2.15      ,  2.08571429,  2.0375    ,  2.        ])

In [13]:
abba

array([  3.        ,   4.88888889,   6.77777778,   8.66666667,
        10.55555556,  12.44444444,  14.33333333,  16.22222222,
        18.11111111,  20.        ])

In [15]:
oba

array([  0.        ,   1.11111111,   2.22222222,   3.33333333,
         4.44444444,   5.55555556,   6.66666667,   7.77777778,
         8.88888889,  10.        ])